In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.io import read_image
from torchvision import transforms
from PIL import Image
import math

In [2]:
# Hyperparameters
batch_size = 16
learning_rate = 0.001

image_width = 475
image_height = 100
# Convolutional layer parameters
kernel_size = 3
stride = 1
padding = 1
channels = 32

# Calculate output size after convolutional layer
conv_output_width = math.floor((image_width - kernel_size + 2 * padding) / stride) + 1
conv_output_height = math.floor((image_height - kernel_size + 2 * padding) / stride) + 1

# Calculate output size after max pooling layer
pool_output_width = math.floor(conv_output_width / 2)
pool_output_height = math.floor(conv_output_height / 2)
print(pool_output_width, pool_output_height)

237 50


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(pool_output_width * pool_output_height * channels, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, pool_output_width * pool_output_height * channels)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
def read_image(path):
    return Image.open(path)
# load model
model = CNN()
model.load_state_dict(torch.load('binary_classifier_cnn_words.pth'))

# load our and foreign image:
our_image = read_image('./our1.png')
foreign_image = read_image('./foreign1.png')

# convert to tensor
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# transform images
our_image = transform(our_image)
foreign_image = transform(foreign_image)

# evaluate images by model
our_image = our_image.unsqueeze(0)
foreign_image = foreign_image.unsqueeze(0)
our_output = model(our_image)
foreign_output = model(foreign_image)

# evaluate output
our_output = torch.squeeze(our_output)
foreign_output = torch.squeeze(foreign_output)
our_output = (our_output > 0).int()
foreign_output = (foreign_output > 0).int()


# print results
print(f'Our image: {our_output}')
print(f'Foreign image: {foreign_output}')

Our image: 1
Foreign image: 0
